In [1]:
#| default_exp 20_prediction-analysis

In [2]:
%load_ext autoreload
%autoreload 2

from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [5]:
#| export
import pandas as pd
from typing import List
from torch.utils.data import Dataset

from xcai.basics import *
from xcai.analysis import *

from xclib.utils.sparse import retain_topk

from IPython.display import HTML,display

## Helper

In [6]:
#| export
def display_momos(momos_dset:Dataset, oak_dset:Dataset, test_dset:Dataset, idxs:List):
    df = pd.DataFrame([test_dset[i] for i in idxs])

    df = df.rename({'data_input_text':'Document', 'lbl2data_input_text': 'Ground truth labels', 'lnk2data_input_text':'Predicted metadata'}, axis=1)
    momos_df = pd.DataFrame({'MOMOS predictions': [momos_dset[i]['lbl2data_input_text'] for i in idxs]})
    oak_df = pd.DataFrame({'OAK predictions': [oak_dset[i]['lbl2data_input_text'] for i in idxs]})
    
    df = pd.concat([df, momos_df, oak_df], axis=1)
    return df
    

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    topk,num_preds,metric,num_data = 5,10,'P',10
    dataset_name = 'wikiseealsotitles'
    pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/'
    odir_a = '/home/scai/phd/aiz218323/scratch/outputs/64-ngame-ep-for-wikiseealso-with-entropy-loss-1-0'
    odir_b = '/home/scai/phd/aiz218323/scratch/outputs/64-ngame-ep-for-wikiseealso-with-entropy-loss-1-0'

    """ Load data """
    pkl_file = f'{pkl_dir}/processed/{dataset_name}_data-metas_distilbert-base-uncased_xcs.pkl'
    with open(pkl_file, 'rb') as file: block = pickle.load(file)

    """ Load predictions """
    pred_file = f'{odir_a}/predictions/test_predictions.pkl'
    plbl_a = get_pred_sparse(pred_file, block.n_lbl)
    
    pred_file = f'{odir_b}/predictions/test_predictions.pkl'
    plbl_b = get_pred_sparse(pred_file, block.n_lbl)

    """ Analysis """
    pattern = r'^(data|lbl2data|lbl2data_aug_cat)_input_text$'

    pdset_a = TextColumns(get_pred_dset(retain_topk(plbl_a, k=num_preds), block), pat=pattern)
    pdset_b = TextColumns(get_pred_dset(retain_topk(plbl_b, k=num_preds), block), pat=pattern)
    test_dset = TextColumns(block.test.dset, pat=pattern)

    eval_a = pointwise_eval(plbl_a, block.test.dset.data.data_lbl, block.test.dset.data.data_filterer, topk=topk, metric=metric)
    eval_b = pointwise_eval(plbl_b, block.test.dset.data.data_lbl, block.test.dset.data.data_filterer, topk=topk, metric=metric)
    eval_a = np.array(eval_a.sum(axis=1)).squeeze()
    eval_b = np.array(eval_b.sum(axis=1)).squeeze()
    idxs = np.argsort(eval_b - eval_a)[:num_data]

    display(HTML(compare_text(pdset_a, pdset_b, test_dset, idxs)))
    

In [ ]:
#| export
if __name__ == '__main__':
    topk,num_preds,metric = 5,10,'P'
    dataset_name = 'wikiseealsotitles'
    pkl_dir = '/home/scai/phd/aiz218323/scratch/datasets/'
    odir_a = '/home/scai/phd/aiz218323/scratch/outputs/64-ngame-ep-for-wikiseealso-with-entropy-loss-1-0'
    odir_b = '/home/scai/phd/aiz218323/scratch/outputs/64-ngame-ep-for-wikiseealso-with-entropy-loss-1-0'

    output_file = 'output.csv'

    """ Load data """
    pkl_file = f'{pkl_dir}/processed/{dataset_name}_data-lnk_distilbert-base-uncased_xcs.pkl'
    with open(pkl_file, 'rb') as file: block = pickle.load(file)

    data_meta = retain_topk(block.test.dset.meta.lnk_meta.data_meta, k=3)
    lbl_meta = block.test.dset.meta.lnk_meta.lbl_meta
    block.test.dset.meta.lnk_meta.update_meta_matrix(data_meta, lbl_meta)

    """ Load predictions """
    pred_file = f'{odir_a}/predictions/test_predictions.pkl'
    plbl_a = get_pred_sparse(pred_file, block.n_lbl)
    
    pred_file = f'{odir_b}/predictions/test_predictions.pkl'
    plbl_b = get_pred_sparse(pred_file, block.n_lbl)

    """ Analysis """
    pattern = r'^(data|lbl2data|lnk2data)_input_text$'

    pdset_a = TextColumns(get_pred_dset(retain_topk(plbl_a, k=topk), block), pat=pattern)
    pdset_b = TextColumns(get_pred_dset(retain_topk(plbl_b, k=topk), block), pat=pattern)
    test_dset = TextColumns(block.test.dset, pat=pattern)

    eval_a = pointwise_eval(plbl_a, block.test.dset.data.data_lbl, block.test.dset.data.data_filterer, topk=topk, metric=metric)
    eval_b = pointwise_eval(plbl_b, block.test.dset.data.data_lbl, block.test.dset.data.data_filterer, topk=topk, metric=metric)
    
    eval_a = np.array(eval_a.sum(axis=1)).squeeze()
    eval_b = np.array(eval_b.sum(axis=1)).squeeze()
    idxs = np.argsort(eval_b - eval_a)

    df = display_momos(pdset_a, pdset_b, test_dset, idxs)
    df.to_csv(output_file)

    